## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [1]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [2]:
load_dotenv(override = True)
openai = OpenAI()

In [3]:
reader = PdfReader("me/personal_linkedIn.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin = linkedin + text

In [4]:
print(linkedin)

   
Contact
SA 6/128-S Om Nagar Colony Lane
no. 4 Paharia , Varanasi
07355208142 (Home)
siddharthwolverine@gmail.co
m
www.linkedin.com/in/siddharth-
singh-021b34193 (LinkedIn)
Top Skills
Retrieval-Augmented Generation
(RAG)
Semantic Search
FastAPI
Languages
Hindi (Native or Bilingual)
English (Professional Working)
Certifications
Tableau Essential Training
Microsoft Power BI - The Practical
Guide [2023 EDITION]
Data Visualization with Tableau
Specialization
Excel Skills for Business
Specialization
Google Data Analytics Specialization
Siddharth Singh
Researcher at Tata Consultancy Services
Bangalore Rural, Karnataka, India
Summary
I am a technology researcher and problem-solver, passionate about
building scalable, intelligent, and user-friendly solutions. My current
work focuses on Agentic AI applications, where I combine LLMs,
semantic search, and FastAPI/Django/AngularJS to develop systems
that let users query their documents through smart, context-aware
chat interfaces.
Earlier, I co

In [5]:
with open("me/summary.txt", "r", encoding = "utf-8") as f:
    summary = f.read()

In [6]:
name = "Siddharth Singh"

In [7]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt = system_prompt + f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt = system_prompt + f"With this context, please chat with the user, always staying in character as {name}."

In [8]:
system_prompt

"You are acting as Siddharth Singh. You are answering questions on Siddharth Singh's website, particularly questions related to Siddharth Singh's career, background, skills and experience. Your responsibility is to represent Siddharth Singh for interactions on the website as faithfully as possible. You are given a summary of Siddharth Singh's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nHi, myself Siddharth, I am working as a researcher in Tata Consultancy Services (TCS). Before this I worked as a Technical support engineer for a retail client based in Kuwait.\nI like to play football, badminton, interested in playing PC games, love gardening and cooking. I recently have started reading books and enjoying it very much.\n\n## LinkedIn Profile:\n\xa0 \xa0\nContact\nSA 6/128-S Om Nagar Colony Lane\

In [9]:
def chat(message, history):
    messages = [{"role" : "system", "content" : system_prompt}] + history + [{"role" : "user", "content" : message}]
    response = openai.chat.completions.create(model = "gpt-4o-mini", messages = messages)
    return response.choices[0].message.content

## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [10]:
gr.ChatInterface(chat, type = "messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [11]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable : bool
    feedback : str

In [51]:
Evaluation.model_json_schema()

{'properties': {'is_acceptable': {'title': 'Is Acceptable', 'type': 'boolean'},
  'feedback': {'title': 'Feedback', 'type': 'string'}},
 'required': ['is_acceptable', 'feedback'],
 'title': 'Evaluation',
 'type': 'object'}

In [52]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt = evaluator_system_prompt + f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt = evaluator_system_prompt + "With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."
evaluator_system_prompt = evaluator_system_prompt + f"Respond back in the given schema so that I can parse back it to JSON easily - {Evaluation.model_json_schema()}"

In [53]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt = user_prompt + f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt = user_prompt + f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt = user_prompt + "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [ ]:
import os
gemini = OpenAI(api_key = os.getenv("GOOGLE_API_KEY"), base_url = "https://generativelanguage.googleapis.com/v1beta/openai/")

In [ ]:
import json

def evaluate(reply, message, history) -> Evaluation:
    messages = [{"role" : "system", "content" : evaluator_system_prompt}] + \
        [{"role" : "user", "content" : evaluator_user_prompt(reply, message, history)}]
    response = openai.chat.completions.create(model = "gpt-4o-mini", messages = messages)

    try:
        response_json = json.loads(response.choices[0].message.content)
        return response_json
    except Exception as e:
        # Handle invalid JSON or missing fields
        return Evaluation(is_acceptable = False, feedback = f"LLM response parsing failed: {str(e)}")

In [ ]:
messages = [{"role" : "system", "content" : system_prompt}] + [{"role" : "user", "content" : "do you hold a patent?"}]
response = openai.chat.completions.create(model = "gpt-4o-mini", messages = messages)
reply = response.choices[0].message.content

In [77]:
reply

'As of now, I do not hold a patent. However, during my tenure at Tata Consultancy Services, I have contributed to the development of tools aimed at analyzing patent data and enhancing intellectual property management strategies for organizations. My role primarily focused on improving existing functionality and features rather than filing patents myself. If you have specific questions about my work or projects, feel free to ask!'

In [78]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=False, feedback='LLM response parsing failed: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)')

In [60]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt = updated_system_prompt + f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt = updated_system_prompt + f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role" : "system", "content" : updated_system_prompt}] + history + [{"role" : "user", "content" : message}]
    response = openai.chat.completions.create(model = "gpt-4o-mini", messages = messages)
    return response.choices[0].message.content

In [61]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role" : "system", "content" : system}] + history + [{"role" : "user", "content" : message}]
    response = openai.chat.completions.create(model = "gpt-4o-mini", messages = messages)
    reply = response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [62]:
gr.ChatInterface(chat, type = "messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/home/siddharth/Documents/LLM_Learning/agents/.venv/lib/python3.12/site-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/siddharth/Documents/LLM_Learning/agents/.venv/lib/python3.12/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/siddharth/Documents/LLM_Learning/agents/.venv/lib/python3.12/site-packages/gradio/blocks.py", line 2220, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/siddharth/Documents/LLM_Learning/agents/.venv/lib/python3.12/site-packages/gradio/blocks.py", line 1729, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/siddharth/Documents/LLM_Learning/a

In [ ]:
from pydantic import BaseModel
from google import genai

class Evaluation(BaseModel):
    is_acceptable : bool
    feedback : str

client = genai.Client(api_key = "YOUR_GEMINI_API_KEY")
prompt = "Evaluate this reply..."

response = client.models.generate_content(model = "gemini-2.0-flash-lite", contents = prompt, 
                                          config = {
                                              "response_mime_type" : "application/json", 
                                              "response_schema" : Evaluation  # This sends the schema generated from Pydantic to Gemini
                                              }
)
result : Evaluation = response.parsed  # Directly parse the result as your Pydantic model